In [38]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from utils import generate_filename
import time
from all_continuous_nhanes_schemas import table_schemas

In [4]:
url = 'https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/AUX_J.htm'

r = requests.get(url)

soup = BeautifulSoup(r.text, 'lxml')


In [14]:
tables = ['acculturation_questionnaire',
'albumin_creatinine_urine_laboratory',
'alcohol_use_questionnaire',
'alpha_1_acid_glycoprotein_serum_surplus_laboratory',
'arsenic_total_urine_laboratory',
'arsenics_speciated_urine_laboratory',
'audiometry_acoustic_reflex_examination',
'audiometry_examination',
'audiometry_questionnaire',
'audiometry_wideband_reflectance_examination',
'blood_pressure_cholesterol_questionnaire',
'body_measures_examination',
'cardiovascular_health_questionnaire',
'cholesterol_high_density_lipoprotein_hdl_laboratory',
'cholesterol_low_density_lipoproteins_ldl_triglycerides_laboratory',
'cholesterol_total_laboratory',
'chromium_cobalt_laboratory',
'chromium_urine_laboratory',
'complete_blood_count_with_5_part_differential_in_whole_blood_laboratory',
'complete_blood_count_with_5_part_differential_laboratory',
'consumer_behavior_phone_follow_up_module_adult_questionnaire',
'consumer_behavior_phone_follow_up_module_child_questionnaire',
'consumer_behavior_questionnaire',
'cotinine_and_hydroxycotinine_serum_laboratory',
'current_health_status_questionnaire',
'cytomegalovirus_igg_igm_antibodies_serum_laboratory',
'demographic_variables_and_sample_weights_demographics',
'dermatology_questionnaire',
'diabetes_questionnaire',
'diet_behavior_nutrition_questionnaire',
'dietary_interview_individual_foods_first_day_dietary',
'dietary_interview_individual_foods_second_day_dietary',
'dietary_interview_technical_support_file_food_codes_dietary',
'dietary_interview_total_nutrient_intakes_first_day_dietary',
'dietary_interview_total_nutrient_intakes_second_day_dietary',
'dietary_supplement_use_24_hour_individual_dietary_supplements_first_day_dietary',
'dietary_supplement_use_24_hour_individual_dietary_supplements_second_day_dietary',
'dietary_supplement_use_24_hour_total_dietary_supplements_first_day_dietary',
'dietary_supplement_use_24_hour_total_dietary_supplements_second_day_dietary',
'dietary_supplement_use_30_day_individual_dietary_supplements_dietary',
'dietary_supplement_use_30_day_total_dietary_supplements_dietary',
'disability_questionnaire',
'drug_use_questionnaire',
'early_childhood_questionnaire',
'ethylene_oxide_laboratory',
'fasting_questionnaire_laboratory',
'ferritin_laboratory',
'flame_retardants_urine_laboratory',
'folate_forms_total_individual_serum_laboratory',
'folate_rbc_laboratory',
'food_security_questionnaire',
'glycohemoglobin_laboratory',
'health_insurance_questionnaire',
'hepatitis_a_laboratory',
'hepatitis_b_surface_antibody_laboratory',
'hepatitis_c_rna_hcv_rna_confirmed_antibody_inno_lia_genotype_laboratory',
'hepatitis_e_igg_igm_antibodies_laboratory',
'hepatitis_questionnaire',
'high_sensitivity_c_reactive_protein_laboratory',
'hospital_utilization_access_to_care_questionnaire',
'housing_characteristics_questionnaire',
'immunization_questionnaire',
'income_questionnaire',
'insulin_laboratory',
'iodine_urine_laboratory',
'iron_status_serum_laboratory',
'kidney_conditions_urology_questionnaire',
'lead_cadmium_total_mercury_selenium_manganese_blood_laboratory',
'medical_conditions_questionnaire',
'mental_health_depression_screener_questionnaire',
'mercury_inorganic_ethyl_and_methyl_blood_laboratory',
'mercury_inorganic_urine_laboratory',
'metals_urine_laboratory',
'nhanes_file_metadata',
'nickel_urine_laboratory',
'occupation_questionnaire',
'oral_health_questionnaire',
'osteoporosis_questionnaire',
'perchlorate_nitrate_thiocyanate_urine_laboratory',
'perfluoroalkyl_and_polyfluoroalkyl_substances_laboratory',
'perfluoroalkyl_and_polyfluoroalkyl_substances_surplus_laboratory',
'pesticide_use_questionnaire',
'physical_activity_questionnaire',
'physical_activity_youth_questionnaire',
'physical_functioning_questionnaire',
'plasma_fasting_glucose_laboratory',
'prescription_medications_questionnaire',
'preventive_aspirin_use_questionnaire',
'reproductive_health_questionnaire',
'sleep_disorders_questionnaire',
'smoking_cigarette_use_questionnaire',
'smoking_household_smokers_questionnaire',
'smoking_recent_tobacco_use_questionnaire',
'smoking_secondhand_smoke_exposure_questionnaire',
'standard_biochemistry_profile_laboratory',
'transferrin_receptor_laboratory',
'urine_flow_rate_laboratory',
'urine_pregnancy_test_laboratory',
'vitamin_c_laboratory',
'volatile_organic_compound_voc_metabolites_ii_urine_laboratory',
'volatile_organic_compound_voc_metabolites_ii_urine_surplus_laboratory',
'volatile_organic_compound_voc_metabolites_urine_laboratory',
'volatile_organic_compounds_and_trihalomethanes_mtbe_blood_laboratory',
'volatile_toxicant_questionnaire',
'weight_history_questionnaire',
'weight_history_youth_questionnaire']

In [44]:
table_schema = table_schemas[table_name]
table_schema
column_type_dict = {d['name']:d['type'] for d in table_schema}

In [46]:
# column_type_dict

In [54]:
# sources_yml_seed = """version: 2
# sources: 
#   - name: all_continuous
#     schema: nhanes
#     tables: 
# """

# with open("./dbt_generations/sources.yml",'w') as f: 
#             f.write(sources_yml_seed)

for table_name in tables:
    
    table_schema = table_schemas[table_name]
    column_type_dict = {d['name']:d['type'] for d in table_schema}
    
    try:
        data_df = pd.read_gbq(
            f"""SELECT doc_file_url, start_year, end_year
            FROM nhanes.{table_name}
            WHERE end_year < 2020
            AND doc_file_url IS NOT NULL
            ORDER BY start_year DESC
            LIMIT 2
            """,
            project_id="nhanes-genai",
            dialect="standard",
        )

        url = data_df['doc_file_url'].tolist()[0]
    except Exception as ex:
        print(ex)
        print(table_name)
        data_df = pd.read_gbq(
            f"""SELECT doc_file_url, start_year, end_year
            FROM nhanes.{table_name}
            WHERE doc_file_url IS NOT NULL
            ORDER BY start_year DESC
            LIMIT 2
            """,
            project_id="nhanes-genai",
            dialect="standard",
        )

        url = data_df['doc_file_url'].tolist()[0]
    
    time.sleep(1)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    
    if not soup.find('div',id='Codebook'):
        url = data_df['doc_file_url'].tolist()[1]
        time.sleep(1)
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'lxml')
        if not soup.find('div',id='Codebook'):
            print(f"Unable to find codebook for {table_name}")
            continue

    variable_definitions = []

    for variable_section in soup.find('div',id='Codebook').find_all('div',{'class':'pagebreak'}):
        variable_definition = {}
        variable_name = variable_section.find('h3')['id']
        variable_definition['name'] = variable_name
        for item in list(zip(variable_section.find_all("dt"),variable_section.find_all("dd"))):
            if 'Label' in item[0].text:
                variable_definition['label'] = item[1].text.strip()
                variable_definition['code'] = generate_filename(item[1].text.strip(),'')
            if 'Text' in item[0].text or 'Instructions' in item[0].text:
                variable_definition['description'] = item[1].text.strip()
            targets = []
            if 'Target' in item[0].text:
                targets.append(" ".join(item[1].text.strip().split()))
            
            variable_definition['targets'] = targets


        table = variable_section.find('table',{'class':'values'})
        
        if variable_name in column_type_dict.keys():
            if table:
                headers = []
                for i in table.find_all('th'):
                    title = i.text.strip()
                    headers.append(title)

                data = []

                for j in table.find_all('tr')[1:]:
                    row_data = j.find_all('td')
                    row = [i.text.strip() for i in row_data]
                    data.append(row)

                table_df = pd.DataFrame(columns = headers,data=data)

                if '.' in table_df['Code or Value'].tolist() and 'Range of Values' not in table_df['Value Description'].tolist():
                    data_type = 'float_category'
                    variable_definition['data_type'] = 'float_category'
                elif '.' in table_df['Code or Value'].tolist() and 'Range of Values' in table_df['Value Description'].tolist():
                    data_type = 'float'
                    variable_definition['data_type'] = 'float'
                elif any(['blank' in c for c in table_df['Code or Value'].tolist()]):
                    data_type = 'string'
                    variable_definition['data_type'] = 'string'
                else:
                    data_type = 'unknown'
                    variable_definition['data_type'] = 'unknown'

                sql_case_statement = """CASE
        """
                for code, value, cnt, cumul, skip in table_df.values:
                    if code != '.' and 'blank' not in code:
                        value = value.replace("'","")
                        if data_type == 'float_category':
                            if column_type_dict[variable_name] in ('INTEGER','FLOAT'):
                                sql_case_statement += f"WHEN {variable_name} = {code} THEN '{value}' \n"
                            else:
                                sql_case_statement += f"WHEN {variable_name} = '{code}' THEN '{value}' \n"
                        elif data_type == 'float':
                            if column_type_dict[variable_name] not in ('INTEGER','FLOAT'):
                                sql_case_statement += f"WHEN {variable_name} IS NOT NULL THEN SAFE_CAST({variable_name} AS FLOAT64) \n"
                            else:
                                sql_case_statement += f"WHEN {variable_name} IS NOT NULL THEN {variable_name} \n"
                        else:
                            if column_type_dict[variable_name] in ('STRING'):
                                sql_case_statement += f"WHEN {variable_name} = '{code}' THEN '{value}' \n"
                            else:
                                sql_case_statement += f"WHEN {variable_name} = {code} THEN '{value}' \n"
                    else:
                        sql_case_statement += f"WHEN {variable_name} IS NULL THEN NULL \n"


                sql_case_statement += 'ELSE NULL \n'

                variable_definition['sql'] = sql_case_statement + f" END as {variable_definition['code']}, \n"
            else:
                if variable_definition.get('code',None):
                    variable_definition['sql'] = f"{variable_name} as {variable_definition['code']}, \n"
                else:
                    variable_definition['sql'] = f"{variable_name} as {variable_name}, -- Could not automatically decode \n"
        else:
            continue


        variable_definitions.append(variable_definition)

    stg_alias = f"stg_{table_name}"
    sql = """SELECT
"""

    for d in variable_definitions:
        sql += d['sql'] + '\n'

    sql += f""" FROM {{{{ ref('{stg_alias}') }}}}"""

    # with open(f"./dbt_generations/sources.yml",'a') as f:
    #     f.write(f"      - name: {table_name} \n")

    with open(f"../../dbt/models/all_continuous/{stg_alias}.sql",'w') as f:
        f.write(f"SELECT * FROM nhanes.{table_name}")

    with open(f"../../dbt/models/all_continuous/{table_name}.sql",'w') as f:
        f.write(sql)

list index out of range
complete_blood_count_with_5_part_differential_in_whole_blood_laboratory
Unable to find codebook for dietary_interview_technical_support_file_food_codes_dietary
list index out of range
volatile_organic_compound_voc_metabolites_ii_urine_laboratory


In [24]:
# soup.find('div',id='Codebook')

In [80]:

    
print(sql)

 SELECT
SEQN as respondent_sequence_number,
CASE
WHEN AUAEXSTS = 1 THEN 'Complete' 
WHEN AUAEXSTS = 2 THEN 'Partial' 
WHEN AUAEXSTS = 3 THEN 'Not done' 
WHEN AUAEXSTS IS NULL THEN NULL 
ELSE NULL 
 END as status_of_an_audio_exam,
CASE
WHEN AUAEXCMT = 2 THEN 'SP refusal' 
WHEN AUAEXCMT = 3 THEN 'No time' 
WHEN AUAEXCMT = 4 THEN 'Physical limitation' 
WHEN AUAEXCMT = 5 THEN 'Communication problem' 
WHEN AUAEXCMT = 6 THEN 'Equipment failure' 
WHEN AUAEXCMT = 7 THEN 'SP ill/emergency' 
WHEN AUAEXCMT = 14 THEN 'Interrupted' 
WHEN AUAEXCMT = 51 THEN 'SP unable to comply' 
WHEN AUAEXCMT = 56 THEN 'Came late/left early' 
WHEN AUAEXCMT = 72 THEN 'Error (technician/software/supply)' 
WHEN AUAEXCMT = 99 THEN 'Other, specify' 
WHEN AUAEXCMT = 122 THEN 'Language barrier' 
WHEN AUAEXCMT = 150 THEN 'SP with parent SP' 
WHEN AUAEXCMT IS NULL THEN NULL 
ELSE NULL 
 END as comment_code_for_an_audio_exam,
CASE
WHEN AUQ011 = 1 THEN 'Yes, right ear' 
WHEN AUQ011 = 2 THEN 'Yes, left ear' 
WHEN AUQ011 = 3 TH